In [1]:
import keras
from keras import layers
from keras.layers import Dense, Input, Dropout
from keras.losses import BinaryCrossentropy
from keras import Model
from keras.layers.core.dense import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from keras.backend import dropout
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.callbacks import LearningRateScheduler
import pickle
import pandas as pd
import os
from sklearn.metrics import roc_curve
import numpy as np
import sys
sys.path.append('../')
sys.path.append('ei_python/')
from src.src_utils import sample, fmeasure_score


2023-11-27 16:46:53.917616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 16:46:54.100480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-27 16:46:54.100516: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-27 16:46:54.146699: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 16:46:55.383935: W tensorflow/stream_executor/pla

## Load data

In [2]:
data_file_path = '../data/processed/tadpole_data_imptn_norm.pickle'
with open(data_file_path, 'rb') as file:
    tadpole_data = pickle.load(file)

label_file_path = '../data/processed/tadpole_labels_imptn_norm.pickle'
with open(label_file_path, 'rb') as file:
    tadpole_labels = pickle.load(file)

#Dict to save scores:
score_dict = {'Autoencoder':{'CV':{}, 'Test':{}}, 'XGB':{'CV':{}, 'Test':{}}}

### Concatenate data

In [3]:
problem = 'mci_bl'
data_split = 'train'
concatenated = {}
pd.DataFrame()
for div in ['train', 'test']:
    concatenated[div] = pd.DataFrame()
    for mode in tadpole_data[problem][data_split]:
        temp = pd.DataFrame(tadpole_data[problem][div][mode])
        concatenated[div] = pd.concat([concatenated[div], temp], axis=1)

# Autoencoder

### Functions for autoencoder

In [25]:
def create_ae(layer_size, input_dim,  
              loss_fn, optimizer, dropout_rate=0, lr=.001):

  input_layer = Input(shape = (input_dim, ))
  noise = Dropout(dropout_rate)(input_layer)
  encoder = layers.Dense(layer_size, activation = 'relu')
  encoded = encoder(noise)
  #decoder = Dense(input_dim, activation = 'relu')(encoder)
  decoder = DenseTranspose(encoder, activation='relu')(encoded)
  autoencoder = Model(inputs = input_layer, outputs = decoder)
  autoencoder.compile(loss=loss_fn, 
                      optimizer=keras.optimizers.Adam(
                          learning_rate=lr))

  return autoencoder

def scheduler(epoch, lr):
   if epoch < 45:
     return lr
   else:
     return lr * tf.math.exp(-0.1)
   
def ae_fit_and_predict(ae, input, epochs, batch_size, checkpoint_filepath = '/home/opc/model_checkpoints/model_checkpoints_subfolder/'):
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='loss',
        mode='min',
        save_best_only=True)

    callbacks = LearningRateScheduler(scheduler)

    stack = ae.fit(input, input, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[model_checkpoint_callback])
    ae.load_weights(checkpoint_filepath)
    predict = ae.predict(input, verbose=0)
    #del_checkpoints()
    return predict

def del_checkpoints(checkpoint_filepath = '/home/opc/model_checkpoints/model_checkpoints_subfolder/'):
    for filename in os.listdir(checkpoint_filepath):
        if ('data' in filename) or ('index' in filename) or ('checkpoint' in filename):
            os.remove(checkpoint_filepath + filename)
    return 

def classifier(classifier_input, y, input_dim, epochs, 
               activation='sigmoid'):

  classifier_input_layer = Input(shape = (input_dim, ))
  classifier_act = Dense(1, activation=activation)(classifier_input_layer)
  classifier = Model(inputs = classifier_input_layer, outputs=classifier_act)
  classifier.compile(loss='binary_crossentropy', 
                     optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                     metrics=['accuracy'])

  return classifier

# Adapted from 
# https://medium.com/@sahoo.puspanjali58/a-beginners-guide-to-build-stacked-autoencoder-and-tying-weights-with-it-9daee61eab2b
class DenseTranspose(keras.layers.Layer):
  def __init__(self, dense, activation=None, **kwargs):
      self.dense = dense
      self.activation = keras.activations.get(activation)
      super().__init__(**kwargs)
  def build(self, batch_input_shape):
      self.biases = self.add_weight(name="bias", 
                  initializer="zeros",shape=[self.dense.input_shape[-1]])
      super().build(batch_input_shape)
  def call(self, inputs):
      z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
      return self.activation(z + self.biases)  
  
def ba_max(y_true, y_pred):
    one_minus_specs, sensitivity, thresholds = roc_curve(y_true, y_pred)
    specificity = 1 - one_minus_specs
    balance_accs = (specificity+sensitivity)/2
    np_balance_accs = np.array(balance_accs)
    max_thres = thresholds[np.argmax(np_balance_accs)]
    return max(balance_accs), max_thres

def sens_and_spec(y_true, y_pred):
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    thresh = ba_max(y_true, y_pred)[1]
    y_pred[np.where(y_pred>=thresh)] = 1
    y_pred[np.where(y_pred<thresh)] = 0
    t_pos = len(np.intersect1d(np.where(y_true==1)[0],np.where(y_pred==1)[0]))
    t_neg = len(np.intersect1d(np.where(y_true==0)[0],np.where(y_pred==0)[0]))
    f_pos = len(np.intersect1d(np.where(y_true==0)[0],np.where(y_pred==1)[0]))
    f_neg = len(np.intersect1d(np.where(y_true==1)[0],np.where(y_pred==0)[0]))
    sens = t_pos / (t_pos + f_neg)
    spec = t_neg / (t_neg + f_pos)
    return sens, spec

### Declare initial params

In [26]:
batch_size = 15
optimizer = 'adam'
loss_fn = keras.losses.MeanSquaredError( ) 
epochs = 60
lr_factor = 0.01

x = concatenated['train']
x_test = concatenated['test']

y = tadpole_labels['mci_bl']['train'].map({'MCI': 0, 'DEM': 1})
y = y.reset_index(drop=True)
y_test = tadpole_labels['mci_bl']['test'].map({'MCI': 0, 'DEM': 1})
y_test = y_test.reset_index(drop=True)
#x_train, y = shuffle(x, y)
x_train = x

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 42)

input_size = x_train.shape
input_dim = input_size[1]

### Tune hyperparameters

In [ ]:
all_ae_preds = {}
all_ae_scores = {}
hyperparams = {'initial_tuning': [0.003, 0.001], #'fine_tuning': [.003, .001], 
               'dropout':[0, 0.2, 0.4, 0.6], 'hidden_layer1': [300, 200], 
               'hidden_layer2': [150, 100], 'hidden_layer3': [100, 75, 50]}

y_pred = np.zeros(y.shape)
tuned_hyperparams = {}
tuning_stages = range(4)
for stage in tuning_stages:
    print(f'\n---------- STAGE {stage} ----------\n')
    tuned_hyperparams[stage] = {}
    for param in hyperparams:
        print(f'\n--- PARAMETER = {param} ---\n')
        current_params = {}

        for other_params in hyperparams:
                if (other_params != param) and (stage == 0):
                    if other_params in tuned_hyperparams[stage].keys():
                        current_params[other_params] = tuned_hyperparams[stage][other_params]
                    else:
                        current_params[other_params] = hyperparams[other_params][0]
                if (other_params != param) and (stage > 0):
                    if other_params in tuned_hyperparams[stage].keys():
                        current_params[other_params] = tuned_hyperparams[stage][other_params]
                    else:  
                        current_params[other_params] = tuned_hyperparams[stage-1][other_params]

        sos = []    
        for p, vals in enumerate(hyperparams[param]):
            current_params[param] = vals
            
            acc_per_fold = []
            fold_no = 1
            y_pred = np.zeros(y.shape)

            for train, test in kfold.split(x_train, y): 

                #x_sample, y_sample = sample(
                        #x_train.iloc[train], y.iloc[train], strategy='undersampling', random_state=42)
                x_sample = x_train.iloc[train]
                y_sample = y.iloc[train]

                autoencoder_1 = create_ae(current_params['hidden_layer1'], dropout_rate=current_params['dropout'], 
                                        lr=current_params['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)
                autoencoder_2 = create_ae(current_params['hidden_layer2'], dropout_rate=current_params['dropout'],
                                        lr=current_params['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)
                autoencoder_3 = create_ae(current_params['hidden_layer3'], dropout_rate=current_params['dropout'], 
                                        lr=current_params['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)

                ae2_input = ae_fit_and_predict(autoencoder_1, x_sample, epochs = epochs, batch_size=batch_size)
                ae3_input = ae_fit_and_predict(autoencoder_2, ae2_input, epochs = epochs, batch_size=batch_size)
                classifier_input = ae_fit_and_predict(autoencoder_3, ae3_input, epochs = epochs, batch_size=batch_size)
                classifier_model = classifier(classifier_input, y_sample, epochs = epochs, input_dim = input_dim)
                final_stack = classifier_model.fit(classifier_input, y_sample, epochs=epochs, 
                                            batch_size=batch_size, verbose=0)
                
                scores = classifier_model.evaluate(x_train.iloc[test, :], y[test])
                y_pred[test] = classifier_model.predict(x_train.iloc[test, :], verbose=0).T[0]
                #print(y_pred)

                #acc = [scores[1]]
                #acc_per_fold += [acc]
                #print(f'\nClass imbalance ratio = {100 * max(1-y[train].sum(), y[train].sum())/len(y[train])}\n')
                fold_no += 1

            acc = sum(1 for a,b in zip(y,[round(prediction) for prediction in y_pred]) if a == b) / len(y)
            sos.append(acc)
        tmp = max(sos)
        idx = sos.index(tmp)
        tuned_hyperparams[stage][param] = hyperparams[param][idx]
        print(hyperparams[param])
        print(sos)
        print(f'Best val for {param} is {tuned_hyperparams[stage][param]}')
        current_params[param]

with open(f'AE_params.pkl', 'wb') as f:
    pickle.dump(tuned_hyperparams, f)

# Cross-validation

In [28]:
with open(f'AE_params.pkl', 'rb') as f:
    tuned_hyperparams = pickle.load(f)

y_pred = np.zeros(y.shape)
fold_no = 0
for train, test in kfold.split(x_train, y): 

    #x_sample, y_sample = sample(
            #x_train.iloc[train], y.iloc[train], strategy='undersampling', random_state=42)
    x_sample = x_train.iloc[train]
    y_sample = y.iloc[train]

    autoencoder_1 = create_ae(tuned_hyperparams[3]['hidden_layer1'], dropout_rate=tuned_hyperparams[3]['dropout'], 
                            lr=tuned_hyperparams[3]['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)
    autoencoder_2 = create_ae(tuned_hyperparams[3]['hidden_layer2'], dropout_rate=tuned_hyperparams[3]['dropout'],
                            lr=tuned_hyperparams[3]['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)
    autoencoder_3 = create_ae(tuned_hyperparams[3]['hidden_layer3'], dropout_rate=tuned_hyperparams[3]['dropout'], 
                            lr=tuned_hyperparams[3]['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)

    ae2_input = ae_fit_and_predict(autoencoder_1, x_sample, epochs = epochs, batch_size=batch_size)
    ae3_input = ae_fit_and_predict(autoencoder_2, ae2_input, epochs = epochs, batch_size=batch_size)
    classifier_input = ae_fit_and_predict(autoencoder_3, ae3_input, epochs = epochs, batch_size=batch_size)
    classifier_model = classifier(classifier_input, y_sample, epochs = epochs, input_dim = input_dim)
    final_stack = classifier_model.fit(classifier_input, y_sample, epochs=epochs, 
                                batch_size=batch_size, verbose=0)
    
    scores = classifier_model.evaluate(x_train.iloc[test], y.iloc[test])
    y_pred[test] = classifier_model.predict(x_train.iloc[test], verbose=0).T[0]

    for layer in classifier_model.layers:
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.assign(tf.zeros_like(layer.kernel))

    
    fold_no += 1

5/5 [==============================] - 0s 2ms/step - loss: 0.4542 - accuracy: 0.7761


In [29]:
from sklearn.metrics import roc_auc_score
# Print scores
fmax_dict = fmeasure_score(y, y_pred)
fmax = fmax_dict['F']
thres = fmax_dict['thres']
f = fmeasure_score(y, y_pred, thres=1-thres, pos_label=0)['F']
auc = roc_auc_score(y, y_pred)
cv_scores = {'Fmax':fmax, 'f (majority)':f, 'AUC':auc, 'thres':thres}
print(f'CV scores:\nfmax (minority): {fmax}\nAUC: {auc}\nf (majority): {f}')
score_dict['Autoencoder']['CV']['fmax (minority)'] = fmax
score_dict['Autoencoder']['CV']['AUC'] = auc
score_dict['Autoencoder']['CV']['f (majority)'] = f

CV scores:
fmax (minority): 0.6599664991624791
AUC: 0.8090431456819818
f (majority): 0.7282463186077643


# Test

In [36]:
with open(f'AE_params.pkl', 'rb') as f:
    tuned_hyperparams = pickle.load(f)

y_test_pred = np.zeros(y_test.shape)

autoencoder_1 = create_ae(tuned_hyperparams[3]['hidden_layer1'], dropout_rate=tuned_hyperparams[3]['dropout'], 
                        lr=tuned_hyperparams[3]['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)
autoencoder_2 = create_ae(tuned_hyperparams[3]['hidden_layer2'], dropout_rate=tuned_hyperparams[3]['dropout'],
                        lr=tuned_hyperparams[3]['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)
autoencoder_3 = create_ae(tuned_hyperparams[3]['hidden_layer3'], dropout_rate=tuned_hyperparams[3]['dropout'], 
                        lr=tuned_hyperparams[3]['initial_tuning'], input_dim = input_dim, loss_fn = loss_fn, optimizer = optimizer)

ae2_input = ae_fit_and_predict(autoencoder_1, x_train, epochs = epochs, batch_size=batch_size)
ae3_input = ae_fit_and_predict(autoencoder_2, ae2_input, epochs = epochs, batch_size=batch_size)
classifier_input = ae_fit_and_predict(autoencoder_3, ae3_input, epochs = epochs, batch_size=batch_size)
classifier_model = classifier(classifier_input, y_sample, epochs = epochs, input_dim = input_dim)
final_stack = classifier_model.fit(classifier_input, y, epochs=epochs, 
                            batch_size=batch_size, verbose=0)

#scores = classifier_model.evaluate(x_test, y_test)
y_test_pred = classifier_model.predict(x_test, verbose=0).T[0]

In [37]:
# Score
f_dict = fmeasure_score(y_test, y_test_pred, thres = cv_scores['thres'])
fminority = f_dict['F']
fmajority = fmeasure_score(y_test, y_test_pred, thres=1-cv_scores['thres'], pos_label=0)['F']
auc = roc_auc_score(y_test, y_test_pred)
test_scores = {'Fmax':fminority, 'f (majority)':fmajority, 'AUC':auc}
print(f'Test scores:\nf (minority): {fminority}\nAUC: {auc}\nf (majority): {fmajority}')
score_dict['Autoencoder']['Test']['f (minority)'] = fminority
score_dict['Autoencoder']['Test']['AUC'] = auc
score_dict['Autoencoder']['Test']['f (minority)'] = fmajority

Test scores:
f (minority): 0.6119402985074627
AUC: 0.7885338345864661
f (majority): 0.7450980392156864


CV scores: $\\$
fmax (minority): 0.6599664991624791
AUC: 0.8090431456819818
f (majority): 0.7282463186077643

Test scores: $\\$
f (minority): 0.6119402985074627
AUC: 0.7885338345864661
f (majority): 0.7450980392156864

## XGBoost Benchmark

In [4]:
import sys
sys.path.append('ei_python/')
from src.ei_python.ei import EnsembleIntegration,  MeanAggregation, MedianAggregation
from src.ei_python.utils import f_minority_score
from src.src_utils import EI_model_train_and_save, fmeasure_score
from src.ei_python.interpretation import *
import pickle 
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import copy

from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, precision_recall_curve, \
    matthews_corrcoef, precision_recall_fscore_support, make_scorer
import sklearn

### Train XGBoost model

In [5]:
# XGB
problem = 'mci_bl'
data_split = 'train'
est = 100

y = tadpole_labels['mci_bl']['train'].map({'MCI': 0, 'DEM': 1})
y = y.reset_index(drop=True)

x = np.array(concatenated[data_split])
y = np.array(tadpole_labels[problem][data_split].map({'MCI': 0.0, 'DEM': 1.0}))

EI_model_train_and_save(f'XGB_benchmark', single_mode=True, meta_training=False,
                        base_predictors={"XGB": XGBClassifier(n_estimators=est)},
                        mode_dict = x, 
                        y = y.astype(int), train = True, random_state=41, 
                        model_building=True, sampling_strategy=None,
                        n_samples=1)

XGB = EnsembleIntegration.load(f'EI.XGB_benchmark')
for metrics in ['fmax (minority)','AUC',  'f (majority)']:
    score_dict['XGB']['CV'][metrics] = XGB.base_summary['metrics']['unimode'].T[metrics][0]



##################################################################################################
######################################## unimode modality ########################################
################################################################################################## 


Training base predictors and generating data for analysis...
Generating meta training data via nested cross validation...


Training base predictors on outer training sets...

Base predictor training is complete: see "base_summary" attribute for a summary of base predictor performance. Meta training data can be found in "meta_training_data" and "meta_test_data" attributes. Run "train_meta" method for analysis of ensemble algorithms.

Training base predictors and generating data for final ensemble...
Generating meta training data via nested cross validation...
Training base predictors on outer training sets...

Model building: meta training data for the final model has been generated and can be found in the "meta_training_data_final" attribute. Final base predidctors have been saved in the "final_models" attribute.

Saved to EI.XGB_benchmark



### Test XGB model

In [18]:
x_test = concatenated['test']
y_test = np.array(tadpole_labels['mci_bl']['test'].map({'MCI': 0.0, 'DEM': 1.0}))
#y_test = y_test.reset_index(drop=True)

XGB = EnsembleIntegration.load(f'EI.XGB_benchmark')
base_models_list = copy.deepcopy(XGB.final_models["base models"]["unimode"])
XGB_dicts = [dictionary for dictionary in base_models_list if dictionary["model name"] == "XGB"]
XGB_model = pickle.loads(XGB_dicts[0]["pickled model"])
fmax_minor_thresh = XGB.base_summary['thresholds']['unimode']['XGB'][0]
test_preds = XGB_model.predict(x_test)

score_dict['XGB']['Test']['f (minority)'] = fmeasure_score(y_test, test_preds, pos_label=1, thres = fmax_minor_thresh)['F']
score_dict['XGB']['Test']['AUC'] = roc_auc_score(y_test, test_preds)
score_dict['XGB']['Test']['f (majority)'] = fmeasure_score(y_test, test_preds, pos_label=0, thres = 1-fmax_minor_thresh)['F']

In [20]:
score_dict['Autoencoder']['CV']['fmax (minority)'] = 0.6599664991624791
score_dict['Autoencoder']['CV']['AUC'] = 0.8090431456819818
score_dict['Autoencoder']['CV']['f (majority)'] = 0.7282463186077643

score_dict['Autoencoder']['Test']['f (minority)'] = 0.6119402985074627
score_dict['Autoencoder']['Test']['AUC'] = 0.7885338345864661
score_dict['Autoencoder']['Test']['f (majority)'] = 0.7450980392156864

## Save scores

In [22]:
with open('benchmark_scores.pkl', 'wb') as f:
    pickle.dump(score_dict, f)